In [5]:
import requests
import warnings
warnings.filterwarnings("ignore")

In [1]:
# this method fetch data from the backend; triggered by url call
# the ingress route called query_es
import requests
def fetch_data(body, index_name):
    url = 'http://127.0.0.1:9090/query_es'
    response = requests.post(url, json={'body': body, 'index': index_name})
    if response.status_code == 200:
        return response.json()
    else:
        return f"Failed to fetch data: Status code {response.status_code}"


In [2]:
# testing 
query = {"size": 5,"query": {"match_all": {}}}
index = 'bomtest'
fetch_data(body=query, index_name=index)

[{'_id': '0miwXI8BAp0N_qlVyFlp',
  '_index': 'bomtest',
  '_score': 1.0,
  '_source': {'latitude': -37.8,
   'local_date_time_full': '20240509191300',
   'longitude': 145.0,
   'name': 'Melbourne (Olympic Park)'}},
 {'_id': '22iwXI8BAp0N_qlV0FlD',
  '_index': 'bomtest',
  '_score': 1.0,
  '_source': {'latitude': -37.8,
   'local_date_time_full': '20240509190000',
   'longitude': 145.0,
   'name': 'Melbourne (Olympic Park)'}},
 {'_id': '3GiwXI8BAp0N_qlV0Flw',
  '_index': 'bomtest',
  '_score': 1.0,
  '_source': {'latitude': -37.8,
   'local_date_time_full': '20240509183000',
   'longitude': 145.0,
   'name': 'Melbourne (Olympic Park)'}},
 {'_id': '3WiwXI8BAp0N_qlV0Fl2',
  '_index': 'bomtest',
  '_score': 1.0,
  '_source': {'latitude': -37.8,
   'local_date_time_full': '20240509180000',
   'longitude': 145.0,
   'name': 'Melbourne (Olympic Park)'}},
 {'_id': '3miwXI8BAp0N_qlV0Fl7',
  '_index': 'bomtest',
  '_score': 1.0,
  '_source': {'latitude': -37.8,
   'local_date_time_full': '202405